In [1]:
import pandas as pd
import re
from collections import Counter
import heapq

def split_file(log_file_path, chunk_size):#מחלק את הקובץ לחלקחם של 1000
    df = pd.read_excel(log_file_path)  
    chunks = [df.iloc[i:i + chunk_size] for i in range(0, len(df), chunk_size)]
    return chunks

def count_errors(df_chunk,error_dict):#סופר את כמות השגיאות לכל מספר שגיאה

    for index, row in df_chunk.iterrows():
        log_message = str(row[0])  
        error_match = re.search(r"ERR_(\d+)", log_message)

        if error_match:
            error_code = error_match.group(1)    
            error_dict[error_code] = error_dict.get(error_code, 0) + 1   

def get_top_n_errors(error_dict, N):#שימוש ב heap לצורך הוצאת כל ה n שגיאות המקסימליות 
    return heapq.nlargest(N, error_dict.items(), key=lambda x: x[1])
#נכון להשתמש בערימה במקרה כזה כי אני לא צריכה מיון של כל הערימה אלא רק הוצאת n מספרים מקסימלים זמן ריצה nlogk

def process_logs(log_file_path, chunk_size, N):
    chunks = split_file(log_file_path, chunk_size)
    error_dict = {}
    [count_errors(chunk,error_dict) for chunk in chunks]
    return get_top_n_errors(error_dict, N)

log_file_path = r"logs.txt.xlsx"  
chunk_size = 1000   
N = 2  

result = process_logs(log_file_path, chunk_size, N)
print(" קודי השגיאה השכיחים ביותר:", result)


 קודי השגיאה השכיחים ביותר: [('404', 200094), ('400', 200068)]


In [11]:
import pandas as pd
from datetime import datetime
import os

def read_file(file_path):#קורא את הקובץ לכל סוגי הקבצים
    ext = os.path.splitext(file_path)[1]
    if ext == '.csv':
        return pd.read_csv(file_path)
    elif ext == '.parquet':
        return pd.read_parquet(file_path)
    elif ext in ['.xls', '.xlsx']:
        return pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format")

def clean_data(df):#מתקן את הדאטא משגיאות 
    # בדיקה שהעמודות קיימות
    required_columns = {'timestamp', 'value'}
    if not required_columns.issubset(df.columns):
        raise ValueError(f"Missing required columns: {required_columns - set(df.columns)}")
    
    # המרת timestamp לפורמט תקני
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

    # המרת value לערך מספרי (יטפל בטקסטים כמו 'not_a_number')
    df['value'] = pd.to_numeric(df['value'], errors='coerce')

    # הסרת נתונים עם ערכים חסרים או לא תקפים
    df = df.dropna(subset=['timestamp', 'value'])

    # הסרת כפילויות
    df = df.drop_duplicates()

    return df

def split_file_by_day(df):#מחלק את הדאטא לחלקים לפי ימים
    df['date'] = df['timestamp'].dt.date
    chunks = [group.drop(columns='date') for _, group in df.groupby('date')]
    return chunks

def calculate_hourly_avg_per_chunk(chunks):#מחשב ממוצע שעתי
    hourly_avgs = []
    for chunk in chunks:
        chunk['hour'] = chunk['timestamp'].dt.floor('H')
        avg = chunk.groupby('hour')['value'].mean().reset_index()
        avg.columns = ['timestamp', 'average_value']
        hourly_avgs.append(avg)
    return hourly_avgs

def combine_hourly_avgs(hourly_avgs):#מצרף את כל הממוצעים לכל חלקי הדאטא
    final_df = pd.concat(hourly_avgs)
    final_df = final_df.sort_values('timestamp').reset_index(drop=True)
    return final_df

def process_log_file(log_file_path):
    print(f"Processing: {log_file_path}")
    
    df = read_file(log_file_path)
    df = clean_data(df)
    chunks = split_file_by_day(df)
    hourly_avgs = calculate_hourly_avg_per_chunk(chunks)
    final_df = combine_hourly_avgs(hourly_avgs)

    # שמירה לקובץ
    output_path = 'final_hourly_avg.csv'
    final_df.to_csv(output_path, index=False)
    print(f" The final hourly averages have been saved to '{output_path}'.")

if __name__ == '__main__':
    process_log_file('time_series.xlsx')


Processing: time_series.xlsx
 The final hourly averages have been saved to 'final_hourly_avg.csv'.


In [10]:
import os
import pandas as pd
from collections import defaultdict

class StreamingHourlyAverages:#מחלקה לעיבוד זרימת נתונים בזמן אמת
    def __init__(self):
        self.data = defaultdict(list)

    def add_entry(self, timestamp, value):#מוסיף נתון לממוצע
        hour = pd.to_datetime(timestamp).floor('H')
        self.data[hour].append(value)

    def get_averages(self):#מוציא ממוצע עד כה
        return {hour: sum(values) / len(values) for hour, values in self.data.items()}
    
def read_file(file_path):
    ext = os.path.splitext(file_path)[1]
    if ext == '.csv':
        return pd.read_csv(file_path)
    elif ext == '.parquet':
        return pd.read_parquet(file_path)
    elif ext in ['.xls', '.xlsx']:
        return pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format")

def clean_data(df):
    required_columns = {'timestamp', 'value'}
    if not required_columns.issubset(df.columns):
        raise ValueError(f"Missing required columns: {required_columns - set(df.columns)}")
    
    df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    df = df.dropna(subset=['timestamp', 'value'])
    df = df.drop_duplicates()
    return df

file_path = "time_series.xlsx" 
df = read_file(file_path)
df = clean_data(df)

stream_processor = StreamingHourlyAverages()

for _, row in df.iterrows():
    stream_processor.add_entry(row["timestamp"], row["value"])

hourly_averages = stream_processor.get_averages()

output_df = pd.DataFrame([
    {"timestamp": hour, "average": avg}
    for hour, avg in sorted(hourly_averages.items())
])
output_df.to_csv("hourly_averages.csv", index=False)
